<a href="https://colab.research.google.com/github/jakobatgithub/unreverb/blob/main/PreprocessAndSaveData_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#path = '/content/drive/My Drive/dsr_project/data/Anechoic/'
#audiopaths = !ls '/content/drive/My Drive/dsr_project/data/Anechoic/'*'/mono/'*'.wav'
#audiopaths = [x.replace("\'", "") for x in audiopaths]

In [ ]:
path = '/content/drive/My Drive/dsr_project/data/but-czas_v1.0/wavs/'
audiopaths = !ls '/content/drive/My Drive/dsr_project/data/but-czas_v1.0/wavs/F'*'/'*'.wav'
audiopaths = [x.replace("\'", "") for x in audiopaths]
audiopaths

In [ ]:
#irpaths = !ls '/content/drive/My Drive/dsr_project/data/random_IRs/IRF'*'.wav'
#irpaths = [x.replace("\'", "") for x in irpaths]
#irpaths

In [ ]:
irpaths = !ls '/content/drive/My Drive/dsr_project/data/samplicity-ir/'*'Rooms'*[0-1][1-9]*'/'*'M-to-S.wav'
irpaths = [x.replace("\'", "") for x in irpaths]
irpaths

In [ ]:
# this is done here:
#   we load the audio data without reverb - the input
#   we load the impulse response (IR) functions
#   we generate the audio data with reverb by convolving all impulse response functions with all audio data without reverb
#   we transform all data to tensorflow datasets
#   one can solve two different tasks:
#     classification: which impulse response function is used
#     regression: remove the reverb from the input
#   correspondingly, we can generate two different datasets here:
#     one with the labels (consequtive numbers of the IRs) as the targets for classification
#     one with the audios without reverb as the targets (commented out)

import numpy as np
import librosa
import matplotlib.pyplot as plt
from IPython.display import Audio 
from IPython.core.display import display
from numpy.fft import fft, ifft
import tensorflow as tf

# sampling rate for resampling
sample_rate = 2**14

# all audio input should have the same length
# we split all audio data into chunks of length n
# n = 2*sample_rate corresponds to 2 seconds
# correspondingly, all audio data without reverb must be at least 2 seconds long
n = sample_rate

# paths for audio without reverb
audiopaths = ['/content/drive/My Drive/dsr_project/data/HarvardWordList/SIHarvardWordListsFemale.wav',
              '/content/drive/My Drive/dsr_project/data/HarvardWordList/SIHarvardWordListsMale.wav']

# paths to IR functions
#irpaths = [
#           "/content/drive/My Drive/dsr_project/data/r1-nuclear-reactor-hall/b-format/r1_bformat.wav", 
#           "/content/drive/My Drive/dsr_project/data/arthur-sykes-rymer-auditorium-university-york/b-format/s1r2.wav",
#           "/content/drive/My Drive/dsr_project/data/trollers-gill/b-format/dales_site1_1way_bformat.wav"]

# function to convolve IRs with audio
def my_conv(signal, kernel):
  kernel_padded = np.hstack((kernel, np.zeros(len(signal) - len(kernel))))
  convolved_signal = np.real(ifft(fft(kernel_padded)*fft(signal)))
  return convolved_signal

# we number the IRs consecutively
unique_labels = [i for i in range(len(irpaths))]

# placeholder lists
irs = []
audios = []
reverb_audios = []
audio_chunks = []
reverb_audio_chunks = []
audio_chunks_2D = []
all_labels = []
labeled_reverb_audio_chunks = []

# loop over all audiofiles without reverb
for audiopath in audiopaths:
  # load audio without reverb 
  audio, audio_sample_rate = librosa.load(audiopath, sr=sample_rate)
  #print(f"audio.shape[0]: {audio.shape[0]}")

  # collect alls audio without reverb in a list
  audios.append(audio)

  # split audio without reverb into chunks of length n
  audio_chunks_1D = []
  for j in range(0, audio.shape[0] - n, n):
    # a single audio chunk without reverb of length n
    audio_chunk = audio[j:j + n]
    
    # collect audio chunks without reverb in a list
    audio_chunks_1D.append(audio_chunk)
  
  # collect audio chunks without reverb in a 2D list
  audio_chunks_2D.append(audio_chunks_1D)


# we convolve all audios with all IRs
# counter for irpaths
iridx = 0
for irpath in irpaths:
  # load IR functions
  #ir_raw, _ = my_get_ir_sample(irpath, resample=sample_rate)
  ir, IR_sample_rate = librosa.load(irpath, sr=sample_rate)
  
  # shorten all IR functions to the same length
  #ir = ir[:, 0:int(sample_rate*2.0)]
      
  # collect IRs in a list
  irs.append(ir)
    
  # loop over all audios without reverb
  # counter for audios
  audioidx = 0
  for audio in audios:

    try:
    
      # convole audio without reverb with IRs to obtain audio with reverb
      #reverb_audio = my_convolve(ir, audio)
      reverb_audio = my_conv(audio, ir)
      #print(f"reverb_audio.shape[1]: {reverb_audio.shape[1]}")
      
      # collect audios with reverb in a list
      reverb_audios.append(reverb_audio)

      # counter for chunks
      chunkidx = 0
      # split audio with reverb into chunks of length n
      for j in range(0, reverb_audio.shape[0] - n, n):

        # a single audio chunk with reverb of length n
        reverb_audio_chunk = reverb_audio[j:j + n]
        
        # collect audio chunks with reverb in a list
        reverb_audio_chunks.append(reverb_audio_chunk)

        # collect audio chunks without reverb in a list
        audio_chunks.append(audio_chunks_2D[audioidx][chunkidx])
        
        # collect corresponding labels in a list
        all_labels.append(unique_labels[iridx])
        
        # collect tuples audio chunks with reverb and labels in a list
        labeled_reverb_audio_chunk = (reverb_audio_chunk, unique_labels[iridx])
        labeled_reverb_audio_chunks.append(labeled_reverb_audio_chunk)
        chunkidx = chunkidx + 1

    except: print("Failed to convert audio. Signal shorter than IR (?)")
    
    # increase counter for audios
    audioidx = audioidx +1
  
  # increase counter for irpaths
  iridx = iridx + 1

# convert list to numpy arrays
features = np.array(reverb_audio_chunks)
targets = np.array(audio_chunks)
labels = np.array(all_labels)
print(f"features.shape: {features.shape}")
print(f"targets.shape: {targets.shape}")
print(f"labels.shape: {labels.shape}")

# determine size of dataset
dataset_size = labels.shape[0]

# convert numpy arrays to tensorflow datasets
# we can generate two different datasets:
#   one with the labels as the targets
#   and one with the sound without reverb as targets
label_dataset = tf.data.Dataset.from_tensor_slices((features, labels))
target_dataset = tf.data.Dataset.from_tensor_slices((features, targets))

# put it all in one dataset
dataset = tf.data.Dataset.from_tensor_slices((features, labels, targets))

# save the data as tensorflow dataset
path = '/content/drive/My Drive/dsr_project/data/HarvardWordList/'
tf.data.experimental.save(dataset, path + "tf_3_randomIRFs_dataset")

In [ ]:
import tensorflow_datasets as tfds

# load data
# path of the tensorflow dataset 
path = '/content/drive/My Drive/dsr_project/data/HarvardWordList/'

# load the tensorflow dataset
dataset = tf.data.experimental.load(path + "tf_3_randomIRFs_dataset")

# determine size of the dataset
dataset_size = sum(1 for _ in dataset)
print(f"dataset_size: {dataset_size}")

def lambda_1(features, labels, targets):
  return (features, labels)

def lambda_2(features, labels, targets):
  return (features, targets)

def lambda_3(features, labels, targets):
  return labels

# obtain the datasets containing only labels or targets
label_dataset = dataset.map(lambda features, labels, targets: lambda_1(features, labels, targets))
target_dataset = dataset.map(lambda features, labels, targets: lambda_2(features, labels, targets))

# obtain all labels
all_labels = dataset.map(lambda features, labels, targets: lambda_3(features, labels, targets))

# transform all_labels dataset to numpy array
all_labels_np = []
for label in all_labels:
  all_labels_np.append(tfds.as_numpy(label))

all_labels_np = np.array(all_labels_np)

# determine unique labels
unique_labels, counts = np.unique(all_labels_np, return_counts=True)
print(f"unique_labels: {unique_labels}")
#print(counts)

# determine number of unique labels
nr_of_unique_labels = unique_labels.shape[0]
print(f"nr_of_unique_labels: {nr_of_unique_labels}")

# shuffle the dataset before splitting in train and validate!
label_dataset = label_dataset.shuffle(dataset_size)

# split dataset in train and validate
train_fraction = 0.8
validate_dataset_size = int(dataset_size * (1.0-train_fraction)) # 20 percent of dataset_size
train_dataset = label_dataset.skip(validate_dataset_size)
validate_dataset = label_dataset.take(validate_dataset_size)